In [14]:
import cv2
import numpy as np

# Load template and defect images
template_image = cv2.imread(r"E:\4th sem\Maths\project\pcb\DeepPCB-master\PCBData\group12300\12300\12300224_temp.jpg", cv2.IMREAD_GRAYSCALE)
defect_image = cv2.imread(r"E:\4th sem\Maths\project\pcb\DeepPCB-master\PCBData\group12300\12300\12300224_test.jpg", cv2.IMREAD_GRAYSCALE)


# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(template_image, None)
keypoints2, descriptors2 = sift.detectAndCompute(defect_image, None)

# Initialize matcher
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test
good_matches = []
for m, n in matches:
    if m.distance < .95 * n.distance:
        good_matches.append(m)

# Extract matched keypoints
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Estimate perspective transformation using RANSAC
M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 6.0)

# Apply perspective transformation to align defect image
rows, cols = defect_image.shape
defect_image_aligned = cv2.warpPerspective(defect_image, M, (cols, rows))

# Convert keypoints to NumPy array of floats
keypoints2_array = np.array([kp.pt for kp in keypoints2], dtype=np.float32)

# Refine alignment using Lucas-Kanade optical flow
nextPts, status, _ = cv2.calcOpticalFlowPyrLK(defect_image_aligned, template_image, keypoints2_array, None)

# Filter keypoints with valid flow
good_new = nextPts[status[:,0] == 1]
good_old = keypoints2_array[status[:,0] == 1]

# Compute absolute difference between refined aligned defect image and template image
difference_image = cv2.absdiff(template_image, defect_image_aligned)

# Apply threshold to highlight significant differences
threshold = 50  # Adjust threshold as needed
difference_image[difference_image < threshold] = 0
difference_image[difference_image >= threshold] = 255


difference_image=difference_image.astype(np.uint8)
# cv2.imwrite("final.jpg",difference_image.astype(np.uint8))


In [15]:

# Convert the difference image to BGR
difference_image_bgr = cv2.cvtColor(difference_image, cv2.COLOR_GRAY2BGR)

# Convert the difference image to HSV color space
hsv = cv2.cvtColor(difference_image_bgr, cv2.COLOR_BGR2HSV)

# Define lower and upper bounds for white color
lower_white = np.array([0, 0, 200])
upper_white = np.array([180, 255, 255])

# Threshold the image to get only white colors
mask = cv2.inRange(hsv, lower_white, upper_white)

# Replace white pixels with red
difference_image_bgr[mask == 255] = [0, 0, 255]


In [16]:

# Apply Canny edge detection
edges = cv2.Canny(template_image, 100, 200)  # Adjust thresholds as needed


In [17]:
# Convert edges image to three channels
edges_bgr = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

# Add the two images
result_image = cv2.add(difference_image_bgr, edges_bgr)

# Display the result
cv2.imshow('Result Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
